In [7]:
!pip install pydub
!pip install ffmpeg-python
!pip install soundfile

In [4]:
import os

def rename_files(directory_path, label):
    # Get a list of files in the directory
    files = os.listdir(directory_path)
    
    # Initialize a counter for numbering the files
    counter = 1
    
    # Iterate through the files and rename them
    for filename in files:
        # Check if the file is an mp3 file
        if filename.endswith('.mp3'):
            # Construct the new filename
            new_filename = f"{label}_song_{counter}.mp3"
            
            # Get the full paths
            old_file = os.path.join(directory_path, filename)
            new_file = os.path.join(directory_path, new_filename)
            
            # Rename the file
            os.rename(old_file, new_file)
            
            # Increment the counter
            counter += 1

# Define the path to your female dataset directory
female_directory_path = r'D:\Tag\Vocal Gender Classifier\Dataset\Female'

# Call the function to rename files
rename_files(female_directory_path, 'female')


In [5]:
import os
from pydub import AudioSegment

def trim_and_rename_files(directory_path, label, duration_ms=30000):
    # Get a list of files in the directory
    files = os.listdir(directory_path)
    
    # Initialize a counter for numbering the files
    counter = 1
    
    # Iterate through the files and process them
    for filename in files:
        # Check if the file is an mp3 file
        if filename.endswith('.mp3'):
            # Construct the new filename
            new_filename = f"{label}_song_{counter}.mp3"
            
            # Get the full paths
            old_file = os.path.join(directory_path, filename)
            new_file = os.path.join(directory_path, new_filename)
            
            # Load the audio file
            audio = AudioSegment.from_mp3(old_file)
            
            # Trim the audio to the specified duration
            trimmed_audio = audio[:duration_ms]
            
            # Export the trimmed audio with the new filename
            trimmed_audio.export(new_file, format="mp3")
            
            # Remove the old file to clean up
            os.remove(old_file)
            
            # Increment the counter
            counter += 1

# Define the path to your male dataset directory
male_directory_path = r'path\to\male\dataset'

# Call the function to trim, rename, and remove original files
trim_and_rename_files(male_directory_path, 'male')


In [8]:
import numpy as np
import librosa
import os
import soundfile as sf  # Ensure you have soundfile installed

def augment_audio(file_path, output_dir):
    # Load audio file
    audio, sr = librosa.load(file_path, sr=None)

    # Pitch shifting
    pitch_shifted_up = librosa.effects.pitch_shift(audio, n_steps=4, sr=sr)
    pitch_shifted_down = librosa.effects.pitch_shift(audio, n_steps=-4, sr=sr)

    # Time stretching
    time_stretched_fast = librosa.effects.time_stretch(audio, rate=1.2)
    time_stretched_slow = librosa.effects.time_stretch(audio, rate=0.8)

    # Save augmented files
    base_filename = os.path.basename(file_path)
    for aug_audio, suffix in zip(
        [pitch_shifted_up, pitch_shifted_down, time_stretched_fast, time_stretched_slow],
        ['_pitch_up', '_pitch_down', '_time_fast', '_time_slow']
    ):
        output_file = os.path.join(output_dir, f"{base_filename[:-4]}{suffix}.wav")  # Change to .wav for compatibility
        sf.write(output_file, aug_audio, sr)  # Use soundfile to save

def augment_dataset(input_dir, output_dir):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate through all files in the input directory
    for subdir, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.mp3'):  # Change this to match your audio file format
                file_path = os.path.join(subdir, file)
                # Check if the file is in the male or female dataset
                if 'male' in file.lower() or 'female' in file.lower():
                    # Create output directory structure
                    relative_path = os.path.relpath(subdir, input_dir)
                    augmented_output_dir = os.path.join(output_dir, relative_path)
                    if not os.path.exists(augmented_output_dir):
                        os.makedirs(augmented_output_dir)

                    # Augment the audio file
                    augment_audio(file_path, augmented_output_dir)

# Example usage
input_directory = r'D:/Tag/Vocal Gender Classifier/Dataset'  # Replace with the path to your male and female datasets
output_directory = r'D:/Tag/Vocal Gender Classifier/Augmented Files'      # Replace with the path to save augmented files

augment_dataset(input_directory, output_directory)
